In [2]:
import random
import math

# Define a simple environment
class SimpleEnv:
    def __init__(self):
        self.state = 2  # Starting in the middle of the 1D grid
        self.goal = 4   # The goal position
        self.states = [0, 1, 2, 3, 4]  # 1D grid of 5 positions

    def reset(self):
        self.state = 2  # Reset to the start position
        return self.state

    def step(self, action):
        if action == 0:  # Move left
            self.state = max(0, self.state - 1)
        elif action == 1:  # Move right
            self.state = min(4, self.state + 1)

        if self.state == self.goal:
            reward = 1  # Goal reached
            done = True
        else:
            reward = 0  # No reward otherwise
            done = False

        return self.state, reward, done


# Neural Network from scratch
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights randomly
        self.weights1 = [[random.random() for _ in range(hidden_size)] for _ in range(input_size)]  # Input to hidden
        self.weights2 = [[random.random() for _ in range(output_size)] for _ in range(hidden_size)]  # Hidden to output
        self.bias1 = [random.random() for _ in range(hidden_size)]  # Bias for hidden layer
        self.bias2 = [random.random() for _ in range(output_size)]  # Bias for output layer

    def relu(self, x):
        return [max(0, xi) for xi in x]  # ReLU activation function

    def relu_derivative(self, x):
        return [1 if xi > 0 else 0 for xi in x]  # Derivative of ReLU

    def forward(self, state):
        # Forward pass: state -> hidden -> output
        self.input = state
        self.hidden = self.relu([sum([self.input[i] * self.weights1[i][j] for i in range(len(self.input))]) + self.bias1[j] for j in range(len(self.weights1[0]))])
        self.output = [sum([self.hidden[i] * self.weights2[i][j] for i in range(len(self.hidden))]) + self.bias2[j] for j in range(len(self.weights2[0]))]
        return self.output

    def backward(self, target, learning_rate):
        # Backpropagation (basic implementation)
        output_error = [(target[i] - self.output[i]) for i in range(len(target))]  # Error in output
        output_delta = output_error  # Assuming derivative of linear output layer is 1

        hidden_error = [sum([output_delta[j] * self.weights2[i][j] for j in range(len(output_delta))]) for i in range(len(self.hidden))]  # Error in hidden layer
        hidden_delta = [hidden_error[i] * self.relu_derivative([self.hidden[i]])[0] for i in range(len(hidden_error))]

        # Update weights and biases (Gradient Descent)
        for i in range(len(self.weights1)):
            for j in range(len(self.weights1[0])):
                self.weights1[i][j] += learning_rate * hidden_delta[j] * self.input[i]
        for i in range(len(self.weights2)):
            for j in range(len(self.weights2[0])):
                self.weights2[i][j] += learning_rate * output_delta[j] * self.hidden[i]

        for j in range(len(self.bias1)):
            self.bias1[j] += learning_rate * hidden_delta[j]
        for j in range(len(self.bias2)):
            self.bias2[j] += learning_rate * output_delta[j]


# Initialize neural networks for prediction and target
input_size = 5  # The size of the state space (one-hot encoded state)
hidden_size = 10  # Number of neurons in the hidden layer
output_size = 2  # Two possible actions (left or right)

prediction_network = NeuralNetwork(input_size, hidden_size, output_size)
target_network = NeuralNetwork(input_size, hidden_size, output_size)  # Target network initialized the same way

# Function to copy weights from prediction to target network
def update_target_network():
    target_network.weights1 = [row[:] for row in prediction_network.weights1]
    target_network.weights2 = [row[:] for row in prediction_network.weights2]
    target_network.bias1 = prediction_network.bias1[:]
    target_network.bias2 = prediction_network.bias2[:]

# One-hot encode the state
def one_hot(state, size):
    vec = [0] * size
    vec[state] = 1
    return vec

# Hyperparameters
alpha = 0.01  # Learning rate for neural network
gamma = 0.9   # Discount factor
epsilon = 1.0  # Exploration rate (starts high, will decay)
epsilon_min = 0.01
epsilon_decay = 0.995
num_episodes = 1000
target_update_freq = 10  # Update target network every 10 episodes

# Function to choose action based on epsilon-greedy policy
def choose_action(state):
    if random.random() < epsilon:  # Explore
        return random.choice([0, 1])  # Random action
    else:  # Exploit
        q_values = prediction_network.forward(one_hot(state, input_size))
        return q_values.index(max(q_values))  # Choose action with the highest Q-value

# Function to calculate target Q-value
def calculate_target(reward, next_state, done):
    if done:
        return reward  # No future rewards if done
    else:
        next_q_values = target_network.forward(one_hot(next_state, input_size))
        return reward + gamma * max(next_q_values)  # Bellman equation

# Training the agent with neural networks
env = SimpleEnv()

for episode in range(num_episodes):
    state = env.reset()
    done = False
    while not done:
        action = choose_action(state)
        next_state, reward, done = env.step(action)

        # Calculate target Q-value
        target_q = prediction_network.forward(one_hot(state, input_size))
        target_q[action] = calculate_target(reward, next_state, done)  # Update the Q-value for the action taken

        # Train the prediction network using backpropagation
        prediction_network.backward(target_q, alpha)

        state = next_state

    # Update target network every few episodes
    if episode % target_update_freq == 0:
        update_target_network()

    # Decay epsilon to reduce exploration over time
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

# After training, print the learned Q-values for each state
for state in range(5):
    print(f"State {state}: {prediction_network.forward(one_hot(state, input_size))}")


State 0: [4.105260622577344, 5.509370241111424]
State 1: [4.178621597284838, 5.6777461169403]
State 2: [3.5691402871591418, 4.5085546634293685]
State 3: [3.8566815090198974, 4.986245968318332]
State 4: [4.0941129130097345, 5.429921617365218]
